# Tutorial - Pore pressure dissipation

Since v0.8.0, ``groundhog`` includes a finite difference routine to calculate pore pressure dissipation. The class ``ConsolidationCalculation`` contains the necessary functionality and allows for 1-way and 2-way drainage conditions. The coefficient of consolidation can be varied over the analysis depth.

In [ ]:
from groundhog.consolidation.dissipation.onedimensionalconsolidation import ConsolidationCalculation
calc = ConsolidationCalculation(height=1, total_time=1e4, no_nodes=21)
calc.set_cv(cv=np.array([0.3, 0.3]), cv_depths=np.array([0, 1]), uniform=False)
calc.set_top_boundary(freedrainage=True)
calc.set_bottom_boundary(freedrainage=False)
calc.set_initial(u0=[100, 100], u0_depths=-[0, 1])
calc.set_output_times(output_times=1e4 * np.linspace(0.1, 1, 10))
calc.calculate()
calc.plot_results(plot_title=r'$ c_v = 0.3m^2/yr $')